In [1]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys, os, random, copy, collections, time, re, argparse
sys.stdour = sys.stderr
import numpy as np
from pprint import pprint
from utils import common
from core import models, datasets
from core.vocabularies import WordVocabularyWithEmbedding

from utils.common import dotDict
from main import Manager
sys.stdout = sys.stderr
args = dotDict({
    'checkpoint_path': 'checkpoints/tmp',
    'mode': 'train',
    'config_path': 'configs/config',
    'cleanup': True,
    'log_file': None,
    'interactive': True,
})

graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
sess.as_default()
manager = Manager(args, sess)
config = manager.config

In [3]:
vocab = WordVocabularyWithEmbedding(config.embeddings, vocab_size=config.vocab_size, lowercase=config.lowercase)


Loading word embeddings from dataset/embeddings/glove.840B.300d.txt.filtered...
Done loading word embeddings.


In [8]:
dataset_type = getattr(datasets, config.dataset_type)
dataset = dataset_type(config.dataset_path.test, vocab)
sys.stdout = sys.stderr
print dataset.raw_data
# print config.input_max_len, config.output_max_len
# batches = dataset.get_batch(2)
# for i, batch in enumerate(batches):
#     print batch.sources.shape
#     print batch.targets.shape
    

[(['_BOS', 'the', 'insurance', 'value', 'of', '$', '2,500', '?'], ['2,500']), (['_BOS', 'birth', 'records', 'are', '$', '17.00', 'per', 'certified', 'copy', '.'], ['17.00']), (['_BOS', 'agradu', 'would', 'like', 'to', 'offer', 'a', '50', '%', 'off', 'coupon', 'for', '$', '200', 'less', 'than', 'half', 'the', 'cost', 'of', 'the', 'course', 'you', 'are'], ['200']), (['_BOS', 'prices', 'start', 'at', '$', '29.95', 'and', 'vary', 'slightly', 'depending', 'on', 'the', 'internet', 'service', 'provider', '.'], ['29.95']), (['_BOS', 'congress', 'will', 'appropriate', '$', '250', 'billion', 'to', 'begin', 'with', ',', 'another', '$', '100', 'billion', 'can', 'be', 'accessed', '.'], ['250', 'billion']), (['_BOS', 'only', '$', '9', 'us', 'each', '6', 'months', '.'], ['9']), (['_BOS', '$', '180', 'a', 'day', 'during', 'the', 'dry', 'months', '.'], ['180']), (['_BOS', 'the', 'cost', 'is', '$', '7.95', 'per', 'year', '.'], ['7.95']), (['_BOS', 'at', 'the', 'end', 'of', '2003', ',', 'the', 'canadian'

In [ ]:
sources, targets = dataset.symbolized
sys.stdout = sys.stderr
from core.vocabularies import BOS_ID

sources =  tf.keras.preprocessing.sequence.pad_sequences(sources, maxlen=config.input_max_len+1, padding='post', truncating='post', value=BOS_ID)
targets =  tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=config.output_max_len, padding='post', truncating='post', value=BOS_ID)
sources = infer_dataset = tf.data.Dataset.from_tensor_slices(sources)
targets = infer_dataset = tf.data.Dataset.from_tensor_slices(targets)
dataset = tf.data.Dataset.zip((sources, targets))
print dataset.batch(32)
print dir(dataset)

In [ ]:
with tf.variable_scope('', reuse=tf.AUTO_REUSE):
    model = manager.create_model(manager.config, vocab)
    #dataset = dataset_type(config.dataset_path, self.vocab)
    

In [ ]:
import sys
sys.stdout = sys.stderr
from six.moves import xrange # pylint: disable=redefined-builtin

import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow.python.ops.rnn_cell_impl import _linear # tf1.4
encoder_inputs = model.e_inputs_emb
decoder_inputs =model.d_inputs
attention_states = model.attention_states
initial_state = model.e_state
cell = model.d_cell
feed_prev=True
dtype=dtypes.float32
scope=None
print encoder_inputs
print decoder_inputs


In [ ]:

with tf.variable_scope(scope or "point_decoder", reuse=tf.AUTO_REUSE):
    print 'encoder_inputs',encoder_inputs
    print 'decoder_inputs', decoder_inputs
    print 'attention_states', attention_states
    attn_length = attention_states.get_shape()[1].value
    attn_size = attention_states.get_shape()[2].value
    states = [initial_state]
    outputs = []

    attnw = tf.get_variable("AttnW", [1, attn_size, attn_size])
    attention_states = tf.nn.conv1d(attention_states, attnw, 1, 'SAME')
    attnv = tf.get_variable("AttnV", [attn_size])

    def attention(output):
        print 'attention_states', attention_states
        print 'output', output
        y = _linear(output, attn_size, True)
        y = tf.reshape(y, [-1, 1, attn_size])
        attention_vectors = tf.nn.softmax(tf.reduce_sum(attnv * tf.tanh(y + attention_states), axis=2))
        return attention_vectors

    for i, d in enumerate(tf.unstack(decoder_inputs, axis=1)):
        print i
        if i > 0:
            tf.get_variable_scope().reuse_variables()
        pointed_idx = d
        # in testing, inputs to decoder won't be used except the first one.
        if feed_prev and i > 0:
            # take argmax, convert the pointed index into one-hot, and get the pointed encoder_inputs by multiplying and reduce_sum.
            pointed_idx =  tf.argmax(output, axis=1)
        
        pointed_idx = tf.reshape(tf.one_hot(pointed_idx, depth=attn_length), [-1, attn_length, 1]) 
        inp = tf.reduce_sum(encoder_inputs * pointed_idx, axis=1) 
        print 'pointed_idx',pointed_idx
        print 'next_inp', inp
        output, state = cell(inp, states[-1])
        output = attention(output)
        states.append(state)
        outputs.append(output)


In [ ]:
dataset_type = getattr(datasets, config.dataset_type)
dataset = dataset_type(config.dataset_path, vocab)
sources, targets = dataset.symbolized
sources = sources[:3]
sources = [' '.join([str(x) for x in s]) for s in sources]

print sources
dataset = dataset.map(lambda string: tf.string_split([string]).values)
# sources = tf.keras.preprocessing.sequence.pad_sequences(sources, padding='post', truncating='post')
# print sources 
# sources = sources.map(lambda words, size: (table.lookup(words), size))
# print sources

#print tf.data.Dataset.from_tensor_slices(sources)
#print tf.data.Dataset(sources)
#print tf.keras.preprocessing.sequence.pad_sequences(dataset.symbolized)
#tf.data.Dataset.zip(dataset.symbolized)
# for i, (source, target) in enumerate(dataset.symbolized):
#     print source, target
#     print dataset.original_sources[i], [dataset.original_sources[i][t] for t in target]
#     if dataset.targets[i] != ['-'] and not ids:
#         print dataset.texts[i]
#         print dataset.values[i].split(':')[0]
#         print i, dataset.original_sources[i], dataset.targets[i], ids
    
    

In [ ]:
def tokens2ids(words):
    return [vocab.token2id(w) for w in words]

infer_batch = tf.placeholder(tf.string, shape=(None,))
dataset = tf.data.Dataset.from_tensor_slices(infer_batch)
dataset = dataset.map(lambda string: tf.string_split([string]).values)
dataset = dataset.map(lambda words: (words, tf.size(words)))
dataset = dataset.map(lambda words, size: (tokens2ids(words), size))
print dataset